## **HuggingFace readme and configuration scraper**

#Loading requirements

In [1]:
import glob

In [2]:
import torch

deps = ["datasets" ,"sentence-transformers", "rich['jupyter']", "requests"]
if torch.cuda.is_available():
    deps.append("faiss-gpu")
else:
    deps.append("faise-cpu")

In [3]:
# retrival
# relevancy
# serendipity

In [4]:
%%capture
!pip install huggingface_hub
!pip install rich

!pip install {" ".join(deps)} --upgrade
from huggingface_hub import hf_api

import requests
import pandas as pd
import matplotlib.pyplot as plt
import rich
%matplotlib inline
plt.style.use("ggplot")


In [5]:
%load_ext rich

In [6]:
from huggingface_hub import hf_api
from huggingface_hub import HfApi, list_models
import pandas as pd
import re
from rich import print
!pip install tqdm
import os
import re
import concurrent.futures


In [7]:
from requests.exceptions import JSONDecodeError
import concurrent.futures
from huggingface_hub import hf_hub_download
from huggingface_hub import hf_hub_url
from functools import lru_cache
import json
from tqdm.auto import tqdm
from requests.exceptions import JSONDecodeError
import concurrent.futures
from huggingface_hub import ModelCard
import pickle
import pandas as pd

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import csv
import io
import concurrent.futures
from tqdm.auto import tqdm
import os
import re

n = 100


Mounted at /content/gdrive


# API - retreive hub info

In [25]:
# # set paths
general_path = "/content/readme_datachunks/"
# # path_models = "/content/readme_datachunks/models_sept4/"
# # path_dataset= "/content/readme_datachunks/datasets_sept4/"
path_path_spaces = "/content/readme_datachunks/spaces_sept4/"

# #create paths
os.mkdir(general_path)
# os.mkdir(path_models)
# os.mkdir(path_dataset)
os.mkdir(path_path_spaces)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 11>:11                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '/content/readme_datachunks/spaces_sept4/'

In [10]:
api = HfApi()
api

In [11]:
hub_datasets = list(iter(api.list_datasets(limit=1000, filter=None, full=True)))


In [12]:
hub_models = list(iter(api.list_models(limit=None, filter=None, full=True, cardData=True)))
hub_models =  [item for item in hub_models if str(item.siblings).find("README.md")!= -1]

In [13]:
# hub_models = [item for item in hub_models if (item.downloads > 0) and (item.likes >0) ]

In [14]:
# hub_models = [item for item in hub_models if (item.downloads > 0) and (item.likes >0) ]

In [15]:
hub_spaces = list(iter(api.list_spaces(limit=None, filter=None, full=False)))

In [16]:
# print(f'Models fetched: { len(hub_models)}.')
# print(f'Datasets fetched: { len(hub_datasets)}.')
print(f'Spaces fetched: { len(hub_spaces)}.')

Spaces fetched: 152982.

## create basic datasets

In [18]:
def create_dataframe(hub_data_lst):
  hub_item_dict = []
  for hub_type, hub_item in hub_data_lst.items():
      for item in hub_item:
          data = item.__dict__
          data["type"] = hub_type
          hub_item_dict.append(data)
  df = pd.DataFrame.from_dict(hub_item_dict)
  df.set_index('id')
  return df

def save_hub_items_tbl(hub_list , item_name):
    dict_items = {item_name: hub_list}
    df_items = create_dataframe(dict_items)
    filename = item_name+".csv"

    print(filename)
    print(df_items.columns)

    df_items.to_csv("/content/gdrive/MyDrive/final project results/"+filename, index=False)
    print(df_items.shape)


# save_hub_items_tbl(hub_datasets , "data_sept")
# save_hub_items_tbl(hub_models , "model_sept")
save_hub_items_tbl(hub_spaces , "space_sept")

space_sept.csv

Index(['id', 'sha', 'lastModified', 'siblings', 'private', 'author', '_id',
       'cardData', 'likes', 'subdomain', 'sdk', 'tags', 'type'],
      dtype='object')

(152982, 13)

# export readmes and config files

In [19]:
from functools import lru_cache
def has_config(model):
    has_config = False
    files = model.siblings
    for file in files:
        if "config.json" in file.rfilename:
            has_config = True
            return has_config
        else:
            continue

def has_file_in_repo(model,file_name):
    has_file = False
    files = model.siblings
    for file in files:
        if file_name in file.rfilename:
            has_file = True
            return has_file
        else:
            continue

@lru_cache(maxsize=None)
def get_model_labels(model):
    try:
        url = hf_hub_url(repo_id=model.modelId, filename="config.json")
        return model.modelId, list(requests.get(url).json()['label2id'].keys())
    except (KeyError, JSONDecodeError, AttributeError):
        return model.modelId, None

def get_item_readme(model , item_type):
    url = hf_hub_url(repo_id=model , filename="README.md", repo_type=item_type)
    return requests.get(url).text


def get_item_config(model , item_type):
    #url = hf_hub_url(repo_id=model.id , filename="config.json", repo_type=item_type)
    url = hf_hub_url(repo_id=model , filename="config.json", repo_type=item_type)
    return requests.get(url).text

In [20]:
def get_data_model(model):
    readme = get_item_readme(model, "model")
    # config = get_item_config(model, "model")
    return model , readme #, config

def get_data_dataset(model):
    readme = get_item_readme(model, "dataset")
    return model, readme

def get_data_space(model):
    readme = get_item_readme(model,"space" )
    return  model, readme

In [21]:
def get_items_data(item_list, get_data=get_data_model, used_columns=['id','readme','config'],
                   item="itemname", filename="data.csv"):
    with concurrent.futures.ThreadPoolExecutor() as e:
        tasks = []
        results = []
        with tqdm(total=len(item_list)) as progress, \
             io.StringIO() as csv_buffer:
            writer = csv.writer(csv_buffer ,  escapechar='\\')
            writer.writerow(used_columns)

            for model in item_list:
                future = e.submit(get_data, model)
                future.add_done_callback(lambda p: progress.update())
                tasks.append(future)

            for task in concurrent.futures.as_completed(tasks):
                results.append(task.result())
                writer.writerow(task.result())

            with open(filename, mode='a', newline='', encoding='utf-8') as f:
                f.write(csv_buffer.getvalue())

    print("<done>")


# def get_items_data(item_list, get_data=get_data_model, used_columns=['id', 'readme', 'config'],
#                    item="itemname", filename="data.csv"):
#     with concurrent.futures.ThreadPoolExecutor() as e:
#         tasks = []
#         results = []
#         with tqdm(total=len(item_list)) as progress, \
#              io.StringIO() as csv_buffer:
#             writer = csv.writer(csv_buffer,
#                                  delimiter= '\x1F' ,
#                                   quotechar='"',
#                                   escapechar="\\",
#                                   doublequote=False)
#             writer.writerow(used_columns)

#             for model in item_list:
#                 future = e.submit(get_data, model)
#                 future.add_done_callback(lambda p: progress.update())
#                 tasks.append(future)

#             for task in concurrent.futures.as_completed(tasks):
#                 result = task.result()
#                 # Ensure that data is properly encoded as UTF-8 to handle special characters
#                 # writer.writerow([result.get(column, '') for column in used_columns])
#                 writer.writerow(task.result())

#             with open(filename, mode='a', newline='', encoding='utf-8') as f:
#                 f.write(csv_buffer.getvalue())


#     print("<done>")

In [22]:
def divide_chunks(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

def fetch_repo_files(hub_modelss,
                     get_data = get_data_model ,
                     used_columns=['id','readme', 'config'],
                     item = "model", filename="/content/readme_datachunks/models/") :
    item_list = [item.id for item in hub_modelss]
    hub_chunks= list(divide_chunks(item_list, n))
    count = 0
    checkpoint_batch = 0
    directory_files = os.listdir(filename)
    if len(directory_files) >1 :
        numbers_new= [int(re.sub('\D', '0', file)) for file in directory_files]
        checkpoint_batch = max(numbers_new) -1
        print(checkpoint_batch)
        hub_chunks = hub_chunks[checkpoint_batch:]

    print("started at " + str(checkpoint_batch))
    for chunk_models in hub_chunks:
        print("chunk " + str(count+ checkpoint_batch) + " / " +str(len(hub_chunks+ checkpoint_batch)))
        get_items_data(chunk_models, get_data , used_columns=used_columns, item=item , filename= filename+item+"_batch_"+str(count+ checkpoint_batch))
        count= count+1

In [23]:
def concat_csv_files (path_of_files, filename_new ):
    all_files = glob.glob(os.path.join(path_of_files, '*'))
    df = pd.concat((pd.read_csv(f) for f in all_files))
    df.to_csv(filename_new , index=False)


In [24]:
fetch_repo_files(hub_spaces, get_data_space, ['id','readme'] , "space" , "/content/readme_datachunks/spaces_sept4/")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in fetch_repo_files:14                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '/content/readme_datachunks/spaces_sept4/'

In [ ]:
prinl"SDsdfg"

In [ ]:
fetch_repo_files(hub_datasets, get_data_dataset, ['id','readme'] , "dataset" , path_dataset)

started at 0

chunk 0 / 619

  0%|          | 0/100 [00:00<?, ?it/s]

<done>

chunk 1 / 619

...

  0%|          | 0/100 [00:00<?, ?it/s]

<done>

chunk 618 / 619

  0%|          | 0/30 [00:00<?, ?it/s]

<done>

In [ ]:
concat_csv_files (path_dataset, '/content/gdrive/MyDrive/final project results/all_dataset_readmes_sept.csv')

In [ ]:
fetch_repo_files(hub_models, get_data_model, ['id','readme'] , "model" , path_models)

started at 0

chunk 0 / 1974

  0%|          | 0/100 [00:00<?, ?it/s]

<done>

chunk 1 / 1974

...

chunk 1249 / 1974

  0%|          | 0/100 [00:00<?, ?it/s]

<done>

chunk 1250 / 1974

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
concat_csv_files (path_models, '/content/gdrive/MyDrive/final project results/all_models_readme_sept4.csv')

In [ ]:
fetch_repo_files(hub_spaces, get_data_space, ['id','readme'] , "space" , path_path_spaces)

In [ ]:
concat_csv_files (path_path_spaces, '/content/gdrive/MyDrive/final project results/all_spaces_readmes_sept.csv')

In [ ]:
import pandas as pd
data_readmes = pd.read_csv("/content/gdrive/MyDrive/final project results/all_dataset_readmes.csv")
data_readmes

In [ ]:
model_readme = pd.read_csv('/content/gdrive/MyDrive/final project results/all_models_readme_fetched.csv')
model_readme

In [ ]:
spaces_readme = pd.read_csv("/content/gdrive/MyDrive/final project results/all_spaces_readmes.csv")
spaces_readme

In [ ]:
data_readmes["readme"][0]

In [ ]:
data_readmes.iloc[6, 1]

In [ ]:
# import glob

# # Get a list of all files in the folder
# all_files = glob.glob(os.path.join(path_models, '*'))

# # Concatenate all files into a single DataFrame
# df = pd.concat((pd.read_csv(f) for f in all_files))

# # Save the concatenated DataFrame to a CSV file
# df.to_csv('all_models_readme.csv', index=False)

In [ ]:
# import glob

# # Get a list of all files in the folder
# all_files = glob.glob(os.path.join(path_dataset, '*'))

# # Concatenate all files into a single DataFrame
# df = pd.concat((pd.read_csv(f) for f in all_files))

# # Save the concatenated DataFrame to a CSV file
# df.to_csv('/content/gdrive/MyDrive/final project results/all_dataset_readmes.csv', index=False)

In [ ]:
# import glob

# # Get a list of all files in the folder
# all_files = glob.glob(os.path.join(path_path_spaces, '*'))

# # Concatenate all files into a single DataFrame
# df = pd.concat((pd.read_csv(f) for f in all_files))

# # Save the concatenated DataFrame to a CSV file
# df.to_csv('/content/gdrive/MyDrive/final project results/all_spaces_readmes.csv', index=False)